In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt

### Import neuron data

In [3]:
# Read neuron spike data
data = pd.read_csv("data/events0.csv", header=None)
data = [data.iloc[i].values for i in range(len(data))]  # convert to list of numpy arrays

In [4]:
# Obtain useful constants
M = len(data)  # number of neurons

t = data  # pour faciliter les notations apres
for i in range(M):
    t[i] = t[i][np.nonzero(t[i])]  # get rid of arrival times = 0

k = [len(ti) for ti in t]  # number of arrivals of each neuron
T = max(max(t[i]) for i in range(M))  # TODO: Do i need T to be the last arrival, or > last arrival ?

In [22]:
# Plot spike train for all neurons
plotspike, ax = plt.subplots(1, 1, figsize= (10,8))
for i in range(M):
    ax.plot(t[i], np.zeros(len(t[i]))+i, linestyle='', marker='+')
ax.set_xlabel('Spike trains')
ax.set_ylabel('Neurons')

Text(0, 0.5, 'Neurons')

### Test things out with a univariate HP

In [88]:
# Single neuron spikes
tt = data[2]
tt = tt[np.nonzero(tt)]
k = len(tt)  # total number of arrivals

In [89]:
fig, ax = plt.subplots(1, 1, figsize=(10,4))
ax.plot(tt, np.zeros(len(tt)), linestyle='', marker='+')
print('k =',k)

k = 244


In [98]:
# Setup the HP
beta = 1
betainv = 1/beta
def g(t, beta=beta):
    return t * np.exp(-beta*t) * (t >= 0)

In [120]:
# Fill out the discrete likelihood matrix
A = np.zeros((k+1, 2))

# Specify first column
A[:, 0] = 1
A[-1, 0] = -tt[-1]

# Specify second column
for i in range(k):
    A[i, 1] = sum(g(tt[i] - tl) for tl in tt if tl < tt[i])
A[-1, 1] = betainv * sum((tt[-1] - ti + betainv)*np.exp(-beta*(tt[-1] - ti)) - betainv for ti in tt)

In [121]:
# Plot matrix A
fig, ax = plt.subplots(1, 1, figsize=(5,8))
im = ax.imshow(A[50:100, :], cmap='viridis', interpolation='nearest')
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('Matrix Coefficients')
plt.show()

### Testing things out with a multivariate HP

In [6]:
# Setup the HP
beta = 1
betainv = 1/beta
def g(t, beta=beta):
    return t * np.exp(-beta*t) * (t >= 0)

In [19]:
# Compute A^j for j=1,...,M
A = [np.zeros((k[j]+1, M+1)) for j in range(M)]  # all matrices A

B = np.zeros(M)  # to help compute the last row of A^j
for n in range(M):
    B[n] = betainv * sum((T - tnl + betainv)*np.exp(-beta*(T - tnl)) - betainv for tnl in t[n])


# Fill out A^j for every j
for j in range(M):
    # First column computation
    A[j][:, 0] = 1
    A[j][-1, 0] = -T

    # Fill out the rest of the matrix (-1 because last row is filled out separately)
    for i in range(A[j].shape[0] - 1):
        for n in range(M):
            A[j][i, n+1] = sum(g(t[j][i] - tnl) for tnl in t[n] if tnl < t[j][i])

    # Fill out last row
    for n in range(M):
        A[j][-1, n+1] = B[n]

In [56]:
# Plot matrix A (without last row, to keep things to scale)
j = 0

fig, ax = plt.subplots(1, 1, figsize=(5,8))
im = ax.imshow(A[j][0:-2, :], cmap='viridis', interpolation='nearest', extent=[1, M, 0, k[j]/5])
fig.colorbar(im, ax=ax)  # Add a colorbar to show the scale
ax.set_title('Matrix Coefficients')
plt.show()